In [2]:
import os
import numpy as np
import mne
%matplotlib qt
import matplotlib
import matplotlib.pyplot as plt

In [39]:
# Load the data
from mne_bids import (BIDSPath,read_raw_bids)

# path where to save the datasets.
bids_root = "../project/Dataset/ds004147-filtered/"
subject_id = '27'


bids_path = BIDSPath(subject=subject_id,task="casinos",
                     datatype='eeg', suffix='eeg',
                     root=bids_root)

# read the file
raw = read_raw_bids(bids_path)
# fix the annotations readin
# ccs_eeg_utils.read_annotations_core(bids_path,raw)

# raw.load_data()
;

Extracting parameters from ../project/Dataset/ds004147/sub-27/eeg/sub-27_task-casinos_eeg.vhdr...
Setting channel info structure...
Reading events from ../project/Dataset/ds004147/sub-27/eeg/sub-27_task-casinos_events.tsv.
The event "Stimulus" refers to multiple event values. Creating hierarchical event names.
    Renaming event: Stimulus -> Stimulus/S  1
    Renaming event: Stimulus -> Stimulus/S  2
    Renaming event: Stimulus -> Stimulus/S  3
    Renaming event: Stimulus -> Stimulus/S  4
    Renaming event: Stimulus -> Stimulus/S  6
    Renaming event: Stimulus -> Stimulus/S  1
    Renaming event: Stimulus -> Stimulus/S  2
    Renaming event: Stimulus -> Stimulus/S  3
    Renaming event: Stimulus -> Stimulus/S  5
    Renaming event: Stimulus -> Stimulus/S  6
    Renaming event: Stimulus -> Stimulus/S  1
    Renaming event: Stimulus -> Stimulus/S  2
    Renaming event: Stimulus -> Stimulus/S  3
    Renaming event: Stimulus -> Stimulus/S  5
    Renaming event: Stimulus -> Stimulus/S  

''

In [153]:
# annotating points of >40uV difference between samples
raw2 = raw.copy()
raw2.load_data()
raw2.filter(l_freq=0.1,h_freq=50)
# for each channel
# raw[chan] set bad annotation at indices = np.where(abs(np.diff(raw[chan][0][0]))>40e-6)[0]
all_onsets = []
all_durations = []
all_descriptions = []
new_anno = raw2.annotations.copy()
for chan in raw2.ch_names:
    bad_indices = np.where(abs(np.diff(raw[chan][0][0]))>40e-6)[0]
    n_anno = len(bad_indices)
    times = raw[chan][1][bad_indices]
    new_anno = new_anno + mne.Annotations(
        onset=times,  # in seconds
        duration=[0.001]*n_anno,  # in seconds, too
        description=["BAD_" + chan + ": 40uV between samples"]*n_anno,
        orig_time=raw2.annotations.orig_time
    )
print(new_anno)
raw2.set_annotations(new_anno)
raw2.annotations
# mne.viz.plot_raw(raw2, duration=15, lowpass=50, highpass=1, scalings={"eeg":100e-6},n_channels=len(raw.ch_names))

<Annotations | 1697 segments: Comment (1), New Segment (1), Stimulus/S 1 ...>
<Annotations | 232068 segments: C3: 40uV between samples (2428), C4: 40uV ...>


<RawBrainVision | sub-33_task-casinos_eeg.eeg, 31 x 2842380 (2842.4 s), ~672.3 MB, data loaded>

In [ ]:
# plot for inspection - simple raw data
# mne.viz.plot_raw(raw, duration=150,highpass=1,lowpass=50, scalings={"eeg":100e-6},n_channels=len(raw.ch_names))

In [ ]:
# evoked - loading and grand averages + plotting

derivs_path = bids_root+"/derivatives/mne-bids-pipeline/"#"Dataset/ds004147-filtered-update/"

evoked_ave_all = []
cond_list = [ # contrasts between win and loss for each task-cue combination
"Stimulus/S  6 - Stimulus/S  7",
"Stimulus/S 16 - Stimulus/S 17",
"Stimulus/S 26 - Stimulus/S 27",
"Stimulus/S 36 - Stimulus/S 37"
]
sub_list = ["27","28","29","30","31","32","33","34","35","36","37","38"]
grand_avgs = []
# for each condition (task-cue combination), create a list of evoked responses across all subjects and then average
for i in range(0,len(cond_list)):
    print(i)
    for sub in sub_list:
        evoked_ave_all.append([])
        evoked_ave_all[i].append(mne.Evoked(derivs_path+"sub-"+sub+"/eeg/sub-"+sub+"_task-casinos_ave.fif",proj=False,verbose=False,condition=cond_list[i]))
    grand_avgs.append(mne.grand_average(evoked_ave_all[i])) # for this condition, average across subjects
    grand_avgs[i].comment=cond_list[i] # name of the condition/difference curve, so that it shows up in the plot

print(len(sub_list), len(evoked_ave_all), len(grand_avgs)) # just to check if the right number of results come out

mne.viz.plot_compare_evokeds(
    grand_avgs,
    combine="mean", # average erp across channels
    # picks="FCz", # if we want to see only FCz
    time_unit="ms",
)